In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import pandas as pd

# Task 5: A/B-тестирование

Итак... давайте, наконец, выясним, повлияло ли посещение ленты новостей на поведение учащихся. Они в итоге приступили раньше к работе над лабораторным заданием? Помните, что у нас есть две подготовленные таблицы в базе данных: test и control. Мы выполним нечто, похожее на A/B-тестирование.
Чтобы поймать эффект, нам нужно вычислить значение дельты (период времени между датой первого коммита и сроком сдачи лабораторного задания) до того момента, когда учащиеся впервые посетили страницу с лентой новостей, и после этого. Мы должны сделать то же самое и для контрольной группы.

Другими словами, каждый пользователь из тестовой таблицы имеет свою собственную временную метку для первого посещения новостной ленты. Мы хотим вычислить среднее значение дельты (разницу между датой первого коммита и сроком сдачи) до этой временной метки и после нее. Мы сделаем то же самое для пользователей в контрольной группе. Вы можете сказать: «Но они вообще не посещали ленту новостей». Это так, и ранее мы решили использовать среднюю временную метку первого просмотра пользователями тестовой группы для пользователей контрольной группы.

Если значение дельты перед первым посещением ленты новостей значительно отличается от этого показателя после первого посещения в тестовой группе, и мы не видим аналогичного эффекта в контрольной группе, значит, создание страницы с новостной лентой было отличной идеей. Мы можем распространить эту практику на всю группу.

Подробное описание:

- [WITH common_table_expression (Transact-SQL)](https://learn.microsoft.com/ru-ru/sql/t-sql/queries/with-common-table-expression-transact-sql?view=sql-server-ver16)
- [Конструкция WITH в T-SQL или обобщенное табличное выражение (ОТВ)](https://info-comp.ru/obucheniest/495-the-with-in-t-sql-or-common-table-expression.html)
- [Предложение SQL HAVING](https://www.w3schools.com/sql/sql_having.asp)
- [Оператор SQL HAVING - интересующие значения агрегатных функций](https://function-x.ru/sql_having.html#paragraph3)

## 1. Создайте соединение с базой данных с помощью библиотеки sqlite3

In [10]:
import sqlite3
con = sqlite3.connect('/content/drive/MyDrive/School21/day11/data/checking-logs.sqlite')

## 2. Используя только один запрос для каждой из групп, создайте два датафрейма: test_results и control_results со столбцами time и avg_diff и только двумя строками.


- times должно иметь значения before и after.

- avg_diff содержит среднее значение дельты для всех пользователей за период времени до первого посещения ленты новостей каждым из них и после этого.
- Учитываются только те пользователи, для которых имеются наблюдения и before, и after

## 3. Мы по-прежнему не используем лабораторное задание 'project1'

In [11]:
test_results = pd.io.sql.read_sql("""
WITH test_r(times, avg_diff) 
AS
(
    SELECT 'before' as times,
            avg((strftime('%s', test.first_commit_ts) - deadlines.deadlines) /3600) as avg_diff
      FROM test
      LEFT JOIN deadlines
      ON test.labname = deadlines.labs
      WHERE test.labname != 'project1' and
            first_view_ts > first_commit_ts
      GROUP BY test.uid
      HAVING test.uid IN (SELECT DISTINCT uid FROM test WHERE labname != 'project1' and
                                                              first_view_ts < first_commit_ts)
  UNION ALL

    SELECT  'after' as times,
            avg((strftime('%s', test.first_commit_ts) - deadlines.deadlines) /3600) as avg_diff
    FROM test
    LEFT JOIN deadlines
    ON test.labname = deadlines.labs
    WHERE test.labname != 'project1' and
          first_view_ts < first_commit_ts
    GROUP BY test.uid
    HAVING test.uid IN (SELECT DISTINCT uid FROM test WHERE labname != 'project1' and
                                                            first_view_ts > first_commit_ts)
 )

SELECT times, AVG(avg_diff) as avg_diff
FROM test_r
GROUP BY times
 """, con, con.commit())

In [12]:
test_results

,times,avg_diff
0,after,-99.523810
1,before,-66.047619


In [13]:
control_results = pd.io.sql.read_sql("""
WITH test_c(times, avg_diff) AS
(
SELECT 'before' as times,
        avg((strftime('%s', control.first_commit_ts) - deadlines.deadlines) /3600) as avg_diff
  FROM control
  LEFT JOIN deadlines
  ON control.labname = deadlines.labs
  WHERE control.labname != 'project1' and
        first_view_ts > first_commit_ts
  GROUP BY control.uid
  HAVING control.uid IN (SELECT DISTINCT uid FROM control WHERE labname != 'project1' and
                                                          first_view_ts < first_commit_ts)
UNION

  SELECT  'after' as times,
           avg((strftime('%s', control.first_commit_ts) - deadlines.deadlines) /3600) as avg_diff
  FROM control
  LEFT JOIN deadlines
  ON control.labname = deadlines.labs
  WHERE control.labname != 'project1' and
        first_view_ts < first_commit_ts
  GROUP BY control.uid
  HAVING control.uid IN (SELECT DISTINCT uid FROM control WHERE labname != 'project1' and
                                                          first_view_ts > first_commit_ts)
 )
SELECT times, AVG(avg_diff) as avg_diff
FROM test_c
GROUP BY times
 
 
 """, con, con.commit())

In [14]:
control_results

,times,avg_diff
0,after,-99.322222
1,before,-98.033333


## 4. Закройте соединение.

In [15]:
con.close()

## 5. Дайте ответ: оказалось ли предположение верным и влияет ли наличие страницы с новостной лентой на поведение учащихся?

***ОТВЕТ:*** 

- **Предположение оказалось верным**.

  - значение дельты перед первым посещением ленты новостей значительно отличается от этого показателя после первого посещения в тестовой группе:
                    (after	-99.523810
                    before	-66.047619)
  и мы не видим аналогичного эффекта в контрольной группе:
                    (after	-99.322222
                    before	-98.033333)
  
  значит, создание страницы с новостной лентой было отличной идеей. 
  
  Мы можем распространить эту практику на всю группу.